<a href="https://colab.research.google.com/github/KacperGrodecki/image_proccessing/blob/main/kuba_img_text_reading_found_words_analyzing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install pyenchant

In [3]:
!apt install -qq enchant

enchant is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [4]:
!apt-get install myspell-pl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
myspell-pl is already the newest version (20170707-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [23]:
import enchant
import pandas as pd
import cv2
import numpy as np
import re
import matplotlib.pyplot as plt      
import pickle
from os import listdir
from os.path import isfile, join
import gc
import sys
import pickle
import string
import difflib

In [6]:
d = enchant.Dict('PL')

In [7]:
text=pd.read_csv('drive/MyDrive/kuba/odczyt_0_85.csv',index_col=0)

In [8]:
text.tail(20)

,0
13667,nowymi i prasą paxowską; Machowski - nt. kolpo...
13668,"PAX, W dyskusji zebrało głos 15 osób w tym Rei..."
13669,"wicz, Dębrowski, Podsumowania dyskusji dokonał..."
13670,"W rererciech, podsumowaniu orcz dyskusji na uwagę"
13671,zasługują:
13672,"tcena kciolicyzmu w Polsce . Stwierdzano, że:"
13673,"ccena kciolicyzmu w Polsce , Stwierdzano, że:"
13674,część biskupów i duchowieńsiwa wyraża niezadow...
13675,z zarysowsujęcych się możliwości normalizacji ...
13676,między Polską a Watykanem;


In [9]:
#check dictionary correction
string_pl=string.ascii_lowercase+'ąśćęó'

file='drive/MyDrive/kuba/odm.txt'


def make_dict(file):
    with open(file, encoding='UTF8') as f:
        lines = f.readlines()
    dict_pl = []
    for line in lines:
        word=line.split()
        dict_pl.append(word)
 
    return [item.replace(',','') for sublist in dict_pl for item in sublist]

dict_pl=make_dict(file)

def check_pl(text):
    return text in dict_pl 

def dicts_pl():
    dicts=[None]*len(string_pl)
    i=0
    for ch in string_pl:
        dicts[i]=[x for x in dict_pl if x[0]==ch ]
        i+=1
    return dicts
        

dicts=dicts_pl()

key=[None]*len(string_pl)
i=0
for ch in string_pl:
        key[i]=ch
        i+=1

def check_string_pl(a):
    result=0
    a=''.join(filter(whitelist.__contains__, a))
    for word in a.split():
       # print(word,' ',check_pl(word))
        result+=int(check_pl_alfabet(word.lower()))
    if len(a.split())>0:
        return result/len(a.split())
    else:
        return 0

def word_correction(a):
    for word in a.split():
        if len(word)>1:
            if check_string_pl(word.lower())<1:
                for i in range(0,len(word)+1):
                    if i==0:
                        for replace in String_pl:
                            word_new=replace+word[1:]
                            if check_pl_alfabet(word_new):
                                print(word_new)
                    else:
                        for replace in String_pl:
                            word_new=word[:i]+replace+word[i+1:]
                            if check_pl_alfabet(word_new):
                                print(word_new)

In [10]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [11]:
import re

In [12]:
#text='\sdv\bf#'

In [13]:
#re.sub(r'\W+', '', text)

In [14]:
def word_correction(a):
    a=str(a)
    correct=[]
    for word in a.split():
        word=re.sub(r'\W+', '', word)
        if len(word)>1:
            if word.isnumeric():
              continue
            try:
              #print('word',word)
              if int(d.check(word))==1:
                return word
              else:
                #take each letter and
                letter_dict=dicts[ord(word.lower()[0])-97]
                #for each letter change it to 
                for local in letter_dict:
                  score=similar(word.lower(),local)
                  if score>=0.9 and score<1:
                    print('sencence ',a,'word',word,' local ',local,' score ',score)
                #maximum=max([similar(word.lower(),local) for local in letter_dict])
                #print(word,' ',maximum)
                correct.append(maximum)
            except:
              pass
    return np.array(correct)

In [15]:
def word_correction(a):
    a=str(a)
    print('sentence: ',a)
    correct=[]
    new_sentence=''
    correction=0
    for word in a.split():
        word=re.sub(r'\W+', '', word)
        if len(word)>1:
            if word.isnumeric():
              continue
            try:
              #print('word',word)
              if int(d.check(word))==1:
                #print('correct',word)
                new_sentence+=word+' '
                continue
              else:
                correction=1
                letter_dict=dicts[ord(word.lower()[0])-97]
                #print(len(letter_dict))
                locals={}
                scores=[]
                score_high=0
                for local in letter_dict:
                  score=similar(word.lower(),local)
                  #print(score)
                  if score>=0.8 and score<1:
                    locals[local]=score
                    if score>score_high:
                      #print(score,' ',local)
                      score_high=score

                key_list = list(locals.keys())
                val_list = list(locals.values())
                position = val_list.index(score_high)
                local=key_list[position]
                #print(locals[score_high])
                    
                print(word,' change to ',local,' score ',score_high)
                new_sentence+=local+' '
                #maximum=max([similar(word.lower(),local) for local in letter_dict])
                #print(word,' ',maximum)
                correct.append(maximum)
            except:
              pass
    if correction==1:
      print('final ',new_sentence)    
    return new_sentence

In [49]:
def word_correction(a):
    a=str(a)
    print('sentence: ',a)
    new_sentence=''
    correction=0
    for word in a.split():
        word=re.sub(r'\W+', '', word)
        if len(word)>1:
            if word.isnumeric():
              continue
            try:
              #print('word',word)
              if int(d.check(word))==1:
                #print('correct',word)
                new_sentence+=word+' '
                continue
              else:
                #print('bad ',word)
                correction=1
                letter_dict=dicts[ord(word.lower()[0])-97]
                local=difflib.get_close_matches(word, letter_dict)
                #print(word,' change to ',local,' score ')
                new_sentence+=local[0]+' '
                #maximum=max([similar(word.lower(),local) for local in letter_dict])
                #print(word,' ',maximum)
            except:
              pass
    if correction==1:
      print('final ',new_sentence)    
    return new_sentence

In [42]:
word='trwac'
letter_dict=dicts[ord(word.lower()[0])-97]
local=difflib.get_close_matches(word, letter_dict)


In [48]:
x='trwac dom'
word_correction(x)

sentence:  trwac dom
bad  trwac
trwac  change to  ['trwa', 'trwając', 'trwacie']  score 
final  trwa dom 


'trwa dom '

In [ ]:
text.applymap(lambda x: word_correction(x))

sentence:  drive/MyDrive/kuba/zdjecia/fig1.jpg
final  
sentence:  Własność
archiwum
Aktotwórca
Tytuł teczki
Daty skrajne
jednostki
archiwalnej
Stara sygnatura
Sygnatura
archiwalna
final  Własność archiwum aktinotrocha Tytuł teczki Daty skrajne jednostki archiwalnej Stara sygnatura Sygnatura archiwalna 
sentence:  IPN BU 0 639/223
sentence:  KARTA INFORMACYJNA
sentence:  Instytut Pamięci Narodowej w Warszawie
sentence:  Aktotwórca
final  aktinotrocha 
sentence:  MSW
sentence:  Tytuł teczki
sentence:  PAX- materiały dotyczące działalności zrzeszenia
sentence:  Daty skrajne
sentence:  jednostki
sentence:  archiwalnej
sentence:  IPN BU 0639/223
sentence:  Sygnatura
sentence:  archiwalna
sentence:  drive/MyDrive/kuba/zdjecia/fig10.jpg
final  
sentence:  Za ałą EZEHO 639/223
final  Za ałyczą 
sentence:  STRUKTURA I WYKONANIE BUDŻETÓW
sentence:  KATOLICKICH ORGANIZACJI SPOŁECZNYCH :
sentence:  "PAX", ChSS, KIK za rok 1971
final  PAX KIK za rok 
sentence:  STOWARZYSZENIE "PAX"
sentence:  Reali

In [19]:
#np.save('drive/MyDrive/kuba/zapis_numpy.csv',np.array(correct))